In [1]:
import base64
import json
import pandas as pd
import requests
import tensorflow as tf

In [4]:
plant = pd.read_csv("data/plant_growth_data.csv")

plant.head()

,Soil_Type,Sunlight_Hours,Water_Frequency,Fertilizer_Type,Temperature,Humidity,Growth_Milestone
0,loam,5.192294,bi-weekly,chemical,31.719602,61.591861,0
1,sandy,4.033133,weekly,organic,28.919484,52.422276,1
2,loam,8.892769,bi-weekly,none,23.179059,44.660539,0
3,loam,8.241144,bi-weekly,none,18.465886,46.433227,0
4,sandy,8.374043,bi-weekly,organic,18.128741,63.625923,0


In [55]:
test_data = plant.drop(columns=['Growth_Milestone']).iloc[1]
test_data

Soil_Type              sandy
Sunlight_Hours      4.033133
Water_Frequency       weekly
Fertilizer_Type      organic
Temperature        28.919484
Humidity           52.422276
Name: 1, dtype: object

In [56]:
def prepare_json(input_data):
    # Extract inputs from dictionary
    soil_type = input_data['Soil_Type']
    sunlight_hours = input_data['Sunlight_Hours']
    water_frequency = input_data['Water_Frequency']
    fertilizer_type = input_data['Fertilizer_Type']
    temperature = input_data['Temperature']
    humidity = input_data['Humidity']

    # Define the feature specification
    feature_spec = {
        "Soil_Type": tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[bytes(soil_type, "utf-8")])
        ),
        "Sunlight_Hours": tf.train.Feature(
            float_list=tf.train.FloatList(value=[sunlight_hours])
        ),
        "Water_Frequency": tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[bytes(water_frequency, "utf-8")])
        ),
        "Fertilizer_Type": tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[bytes(fertilizer_type, "utf-8")])
        ),
        "Temperature": tf.train.Feature(
            float_list=tf.train.FloatList(value=[temperature])
        ),
        "Humidity": tf.train.Feature(
            float_list=tf.train.FloatList(value=[humidity])
        )
    }

    # Create an Example object
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    )

    # Serialize to string
    example_string = example.SerializeToString()

    # Encode to base64
    encoded_example = base64.b64encode(example_string).decode()

    # Prepare the JSON request
    json_data = {
        "signature_name": "serving_default",
        "instances": [{
            "examples": {
                "b64": encoded_example
            }
        }]
    }

    return json.dumps(json_data)


def make_predictions(input_data):
    json_data = prepare_json(input_data)

    endpoint = "http://localhost:8080/v1/models/plantgrowth-model:predict"
    try:
        response = requests.post(endpoint, json=json.loads(json_data))

        if response.status_code == 200:
            predictions = response.json()["predictions"][0][0]
            prediction_label = "1" if predictions >= 0.5 else "0"
            return {
                "prediction": predictions,
                "label": prediction_label
            }
        else:
            return {"error": f"Failed to get predictions from the model. Status code: {response.status_code}"}
    except requests.exceptions.RequestException as e:
        return {"error": f"Error during request to model server: {str(e)}"}


In [57]:
prediction_result = make_predictions(test_data)
print("Prediction:", prediction_result)

Prediction: {'prediction': 0.725691617, 'label': '1'}


In [60]:
actual_label = plant["Growth_Milestone"][1]
print("Actual Label:", actual_label)

Actual Label: 1
